In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import molpy as mp
from pathlib import Path

data_path = Path("data/case1")

In [2]:
struct_h = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "H.pdb"), "H")
struct_n = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "N.pdb"), "N")
struct_m = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "M.pdb"), "M")
struct_p = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "P.pdb"), "P")
struct_t = mp.Monomer.from_frame(mp.io.read_pdb(data_path / "T.pdb"), "T")

typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
typifier.typify(struct_h, workdir=data_path / "H")
typifier.typify(struct_n, net_charge=-1, workdir=data_path / "N")
typifier.typify(struct_m, net_charge=1, workdir=data_path / "M")
typifier.typify(struct_p, workdir=data_path / "P")
typifier.typify(struct_t, workdir=data_path / "T")


<Monomer: 22 atoms>

In [ ]:
struct_h.def_link_site(
    struct_h["atoms"][11],
    None,
    [struct_h["atoms"][10], struct_h["atoms"][15], struct_h["atoms"][16]],
    "TAIL",
)

struct_n.def_link_site(
    struct_n["atoms"][5],
    None,
    [struct_n["atoms"][7]],
    "HEAD",
)
struct_n.def_link_site(
    struct_n["atoms"][10],
    None,
    [struct_n["atoms"][18]],
    "TAIL",
)

struct_m.def_link_site(
    struct_m["atoms"][5],
    None,
    [struct_m["atoms"][0]],
    "HEAD",
)
struct_m.def_link_site(
    struct_m["atoms"][1],
    None,
    [struct_m["atoms"][3]],
    "TAIL",
)

struct_p.def_link_site(
    struct_p["atoms"][0],
    None,
    [struct_p["atoms"][2], struct_p["atoms"][3], struct_p["atoms"][4]],
    "HEAD",
)
struct_p.def_link_site(
    struct_p["atoms"][1],
    None,
    [struct_p["atoms"][6], struct_p["atoms"][7], struct_p["atoms"][8]],
    "TAIL",
)

struct_t.def_link_site(struct_t["atoms"][1], None, [struct_t["atoms"][4]], "HEAD")

<Monomer: 22 atoms>

In [4]:
reacter = mp.reacter.AmberToolsReacter()
reacter.react(struct_h, data_path)
reacter.react(struct_t, data_path)

Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
                   -rf    residue toplogy file name in prep input f

<Monomer: 22 atoms>

In [5]:
polymerizer = mp.polymerizer.AmberToolsPolymerizer()
polymerizer.polymerize(["H", "T"], {"H": struct_h, "T": struct_t}, data_path, "HT")

-I: Adding /opt/conda/envs/AmberTools25/dat/leap/prep to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/lib to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/parm to search path.
-I: Adding /opt/conda/envs/AmberTools25/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/envs/AmberTools25/dat/leap/parm/gaff.dat
Reading title:
AMBER General Force Field for organic molecules (Version 1.81, May 2017)
----- Source: /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p
----- Source of /opt/conda/envs/AmberTools25/dat/leap/cmd/leaprc.water.tip3p done
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/atomic_ions.lib
Loading library: /opt/conda/envs/AmberTools25/dat/leap/lib/solvents.lib
Loading

FileNotFoundError: [Errno 2] No such file or directory: 'data/case1/HT/HT.pdb'